<a href="https://colab.research.google.com/github/Pravanop/ml4Sci_eval/blob/main/Task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using Dataset 2 

In [ ]:
!pip install pyarrow

In [ ]:
import pyarrow.parquet as pq
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import 
import time
import copy

In [ ]:
class jetDataset(Dataset):

    def __init__(self, path):
      self.table = pq.ParquetFile(path)

    def __len__(self):
        return self.table.num_row_groups

    def __getitem__(self, idx):
        row = self.table.read_row_group(idx)
        jet = row.column('X_jets').to_numpy()
        label = np.array(row.column('y'))
        three_array = []
        for j,m in enumerate(jet[0]):
          arr = []
          for i,k in enumerate(jet[0][j]):
            if i == 0:
              arr = k
              arr = arr.reshape(arr.shape[0],1)
            else:
              k = k.reshape(k.shape[0],1)
              arr = np.concatenate((arr,k),axis=0)
          if j == 0 :
            three_array = arr
          else:
            three_array = np.concatenate((three_array,arr),axis=0)
        
        three_array_reshaped = three_array.reshape(125,125,3)
        x = torch.tensor(three_array_reshaped)
        label = label[0]
        
        sample = (x, label)
        return sample

In [ ]:
run1 = jetDataset('/content/drive/MyDrive/GSOC/Application/ML4Sci/Task 2/QCDToGGQQ_IMGjet_RH1all_jet0_run0_n36272.test.snappy.parquet')
run2 = jetDataset('/content/drive/MyDrive/GSOC/Application/ML4Sci/Task 2/QCDToGGQQ_IMGjet_RH1all_jet0_run1_n47540.test.snappy.parquet')
run3 = jetDataset('/content/drive/MyDrive/GSOC/Application/ML4Sci/Task 2/QCDToGGQQ_IMGjet_RH1all_jet0_run2_n55494.test.snappy.parquet')

In [ ]:
sample = run3[0][1]
sample

0.0

In [ ]:
all_datasets = [run1,run2,run3]
final_dataset = torch.utils.data.ConcatDataset(all_datasets)

In [ ]:
train_size = int(0.8 * len(final_dataset))
test_size = len(final_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(final_dataset, [train_size, test_size])

In [ ]:
datasets = {'train':train_dataset,'val':test_dataset}

In [ ]:
dataloaders = {x: DataLoader(datasets[x], batch_size= 2048, shuffle=True, num_workers=0) for x in ['train','val']}

In [ ]:
dataset_sizes = {x: len(datasets[x]) for x in ['train', 'val']}

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
res_mod = models.resnet34(pretrained=True)
num_ftrs = res_mod.fc.in_features
res_mod.fc = nn.Linear(num_ftrs, 2)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


In [ ]:
for name, child in res_mod.named_children():
    print(name)

conv1
bn1
relu
maxpool
layer1
layer2
layer3
layer4
avgpool
fc


In [ ]:
res_mod = res_mod.to(device)
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(res_mod.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            current_loss = 0.0
            current_corrects = 0

            print('Iterating through data...')

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.permute(0, 3, 1, 2)
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs.float())
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels.long())

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                current_loss += loss.item() * inputs.size(0)
                current_corrects += torch.sum(preds == labels.data)

            epoch_loss = current_loss / dataset_sizes[phase]
            epoch_acc = current_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # Make a copy of the model if the accuracy on the validation set has improved
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_since = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_since // 60, time_since % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

In [ ]:
base_model = train_model(res_mod, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=1)

Epoch 0/0
----------
Iterating through data...


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
